## Day 16

https://adventofcode.com/2022/day/16

In [1]:
def parse16(filename):
    with open(filename) as f:
        data = [ l.strip().split("; ")  for l in f.readlines() ] 
        tunnels = {}
        flow = {}
        for d in data:
            v = d[0].split(" has flow rate=")
            V = v[0].replace("Valve ","")
            F = int(v[1])
            T = d[1].replace("tunnels lead to valves ","").replace("tunnel leads to valve ","").split(", ")
            flow[V] = F
            tunnels[V] = T
        return flow, tunnels

In [79]:
flow, tunnels = parse16("examples/example16.txt")
#flow, tunnels = parse16("../AOC2022inputs/input16.txt")
flow, tunnels

({'AA': 0,
  'BB': 13,
  'CC': 2,
  'DD': 20,
  'EE': 3,
  'FF': 0,
  'GG': 0,
  'HH': 22,
  'II': 0,
  'JJ': 21},
 {'AA': ['DD', 'II', 'BB'],
  'BB': ['CC', 'AA'],
  'CC': ['DD', 'BB'],
  'DD': ['CC', 'AA', 'EE'],
  'EE': ['FF', 'DD'],
  'FF': ['EE', 'GG'],
  'GG': ['FF', 'HH'],
  'HH': ['GG'],
  'II': ['AA', 'JJ'],
  'JJ': ['II']})

In [80]:
from queue import Queue

def tracePath(V1,V2,tunnels):
    '''Simple BFS search of shortest path between valve V1 and V2'''
    path = [V1]
    q = Queue()
    q.put(path)
    visited = [ V1 ]
    while not q.empty():
        path = q.get()
        for v in tunnels[ path[-1] ]:
            if v not in visited:
                if v==V2:
                    return path+[v]
                else:
                    pathnew = list(path)
                    pathnew.append(v)
                    q.put(pathnew)
    print("not found")
    return None

tracePath("BB","JJ",tunnels)

['BB', 'AA', 'II', 'JJ']

In [81]:
# Select valves that have flow
haveFlow = sorted([ v for v,f in flow.items() if f>0 ])
haveFlow

['BB', 'CC', 'DD', 'EE', 'HH', 'JJ']

In [82]:
# Compute shortest distance between valves with flow (the only ones I might have an interest to open)
# If I travel directly from V1 to V2, this is meant to represent the case I pass by the 
# intermediate valves without opening them.
# Addind AA as starting point in position 0.

distance = [ [ 0 ]*(len(haveFlow)+1) for _ in range(len(haveFlow)+1) ]

for i in range(1,len(haveFlow)+1):
    for j in range(i+1,len(haveFlow)+1):
        if i!=j:
            V1 = haveFlow[i-1]
            V2 = haveFlow[j-1]
            p = tracePath(V1,V2,tunnels)
            distance[j][i]=len(p)-1
            distance[i][j]=len(p)-1

for k in range(1,len(haveFlow)+1):
    V1 = "AA"
    V2 = haveFlow[k-1]
    p = tracePath(V1,V2,tunnels)
    distance[0][k]=len(p)-1
    distance[k][0]=len(p)-1

distance

[[0, 1, 2, 1, 2, 5, 2],
 [1, 0, 1, 2, 3, 6, 3],
 [2, 1, 0, 1, 2, 5, 4],
 [1, 2, 1, 0, 1, 4, 3],
 [2, 3, 2, 1, 0, 3, 4],
 [5, 6, 5, 4, 3, 0, 7],
 [2, 3, 4, 3, 4, 7, 0]]

In [83]:
from queue import PriorityQueue

def part1( filename, Tmax = 30 ):
    
    flow, tunnels = parse16(filename)
    
    # Select valves that have flow
    haveFlow = [ v for v,f in flow.items() if f>0 ]

    # Compute distance between valves with flow (the only ones I might have an interest to open)
    # If I travel directly from V1 to V2, this is meant to represent the case I pass by the 
    # intermediate valves without opening them.
    # Adding "AA" at position 0 as starting point.

    distance = [ [ 0 ]*(len(haveFlow)+1) for _ in range(len(haveFlow)+1) ]

    for i in range(1,len(haveFlow)+1):
        for j in range(i+1,len(haveFlow)+1):
            if i!=j:
                V1 = haveFlow[i-1]
                V2 = haveFlow[j-1]
                p = tracePath(V1,V2,tunnels)
                distance[j][i]=len(p)-1
                distance[i][j]=len(p)-1

    for k in range(1,len(haveFlow)+1):
        V1 = "AA"
        V2 = haveFlow[k-1]
        p = tracePath(V1,V2,tunnels)
        distance[0][k]=len(p)-1
        distance[k][0]=len(p)-1
    
    # status is represented by pressure, current valve, elapsed time, list of open valves
    path = (0, "AA", 0, [])
    
    q = Queue()
    q.put(path)
    results = []

    while not q.empty():

        path = q.get()
        P,V,T,OpenV = path

        # max time reached, saving path and moving on
        if T==Tmax:
            results.append(path)
            continue

        # current valve is closed, use minute to open it and stay here
        if V not in OpenV and V!="AA":
            Pnew = P - sum([flow[w] for w in OpenV])
            pathnew = (Pnew,V,T+1,sorted(OpenV+[V]))
            q.put(pathnew)

        # current valve is open (or I am at starting point AA), try to move to another one with flow
        # (at any possible distance, thus possibly skipping some intermediate ones w/o opening them yet)
        else:
            moveToNewValve = False
            for v in haveFlow:
                if v not in OpenV: # avoid loops toward already open valves
                    dT = 0
                    if V=="AA":
                        dT = distance[0][haveFlow.index(v)+1]
                    else:
                        dT = distance[haveFlow.index(V)+1][haveFlow.index(v)+1]
                    Tnew = T+dT
                    if Tnew<=Tmax:
                        Pnew = P - dT*sum([flow[w] for w in OpenV])
                        pathnew = (Pnew,v,Tnew,sorted(OpenV))
                        q.put(pathnew)
                        moveToNewValve = True
            if not moveToNewValve:
                # Could not move to any new valve either because it would take too long 
                # or they are already open, thus stay where I am for another minute
                Tnew = T+1
                if Tnew<=Tmax:
                    Pnew = P - sum([flow[w] for w in OpenV])
                    pathnew = (Pnew,v,Tnew,sorted(OpenV))
                    q.put(pathnew)
    
    best = sorted(results,key=lambda x:x[0])[0]
    return -best[0]

In [84]:
part1("examples/example16.txt")

1651

In [86]:
part1("AOC2022inputs/input16.txt") # 1771 too high

1771